In [2]:
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
import torch
import wandb


In [4]:
!wandb login 57566fbb0e091de2e298a4320d872f9a2b200d12

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\DELL\.netrc


# LOAD DATA

In [8]:
def load_data(batch_size,img_size):
    augmentation = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
    ])

    # Load the dataset
    data_path = "../../nature_12K/inaturalist_12K/"
    train_dataset = datasets.ImageFolder(os.path.join(data_path, 'train'), transform = augmentation)
    test_dataset = datasets.ImageFolder(os.path.join(data_path, 'val'), transform = augmentation)

    
    labels = train_dataset.classes
    trainset, valset = random_split(train_dataset, [8000, 1999])

    train_loader = DataLoader(trainset, batch_size = batch_size)
    val_loader = DataLoader(valset, batch_size = batch_size)
    test_loader = DataLoader(test_dataset, batch_size = batch_size)

    return labels , train_loader, val_loader, test_loader



In [24]:
# CHECK DEVICE (CPU / GPU)
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Currently Using :: ", device)

Currently Using ::  cpu


# **Question - 1**

# **CONVOLUTIONAL NEURAL NETWOK CLASS**

In [25]:
from typing import Any


class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self, PARAM) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.filter_org = PARAM["filter_org"]
        self.filter_num = PARAM["filter_num"]
        self.activation = PARAM["activation"]
        self.con_layers = PARAM["con_layers"]
        self.den_layers = PARAM["dense_layers"]
        self.input_channel = PARAM["input_channel"]
        self.filter_num_list = self.organize_filters(self.filter_org, self.filter_num, self.con_layers)
        self.filter_size_list = PARAM["filter_size"]
        self.act = self.activation_fun(PARAM["activation"])
        self.output_act = self.activation_fun(PARAM["output_activation"])
        self.padding = PARAM["padding"]
        self.stride = PARAM["stride"]
        self.pool_padding = PARAM["pool_padding"]
        self.pool_stride = PARAM["pool_stride"]
        self.dense_output_list = PARAM["dense_output_list"]
        self.image_size = PARAM["image_size"]
        self.pool_filter_size = PARAM["pool_filter_size"]
        self.dropout_list = PARAM["dropout"]
        self.create_con_layers(self.input_channel, self.filter_size_list, self.dense_output_list, self.filter_num_list, self.act, self.pool_filter_size, self.output_act, self.image_size, self.dropout_list)
        
        
    def create_con_layers(self, input_channel, filter_size_list, dense_output_list, filter_num_list, act, pool_filter_size, output_act, image_size, dropout_list):
        self.layers = nn.ModuleList()
        computation = 0
        for i in range(1, self.con_layers+1):
            comp = 0
            layer = nn.Sequential(nn.Conv2d(input_channel, filter_num_list[i-1], filter_size_list[i-1], padding=self.padding, stride=self.stride), act, nn.MaxPool2d(pool_filter_size, padding=self.pool_padding, stride=self.pool_stride), nn.Dropout(dropout_list[i-1]))
            
            image_size = (image_size - filter_size_list[i-1] + 2 * self.padding)//self.stride + 1
            
            comp = ((filter_size_list[i-1] ** 2) * input_channel * (image_size ** 2)*filter_num_list[i-1] + filter_num_list[i-1])
            computation += comp
            image_size = (image_size + 2 * self.pool_padding-(1*(pool_filter_size-1))-1)//self.pool_stride + 1
            print(image_size)
            # print(comp)
            input_channel = filter_num_list[i-1]
            self.layers.append(layer)
        dense_input = filter_num_list[self.con_layers-1] * (image_size ** 2)
        for i in range(1, self.den_layers+1):
            comp = 0
            layer = nn.Sequential(nn.Linear(dense_input, dense_output_list[i-1]), act, nn.Dropout(dropout_list[self.con_layers + i-1]))
            comp = ((dense_input  + 1) * dense_output_list[i-1])
            computation += comp
            dense_input = dense_output_list[i-1]
            self.layers.append(layer)
            # print(computation)
            # print(comp)
        layer = nn.Sequential(nn.Linear(dense_input, 10), output_act)
        comp = ((dense_input  + 1) * 10)
        computation += comp
        # print(comp)
        print("Computation :: ", computation)
        self.layers.append(layer)


    def organize_filters(self, filter_org, filter_number, layers):
        if filter_org == "same":
            filter_num = [filter_number] * layers
        elif filter_org == "double":
            filter_num = [filter_number * (2 ** i) for i in range(layers)]
        elif filter_org == "half":
            filter_num = [int(filter_number * (2 ** (-i))) for i in range(layers)]
        return filter_num
    
    
    def activation_fun(self, act):
        if act == "ReLU":
            act_fun =nn.ReLU()
        elif act == "GELU":
            act_fun = nn.GELU()
        elif act == "SiLU":
            act_fun = nn.SiLU()
        elif act == "Mish":
            act_fun = nn.Mish()
        elif act == "softmax":
            act_fun = nn.Softmax(dim=1)
        elif act == "ELU":
            act_fun = nn.ELU()
        return act_fun
    
    
    def forward(self,x):
        for i in range(0, self.con_layers):
            x = self.layers[i](x)
        x = self.flatten(x)
        for i in range(0, self.den_layers):
            x = self.layers[i+self.con_layers](x)
        x = self.layers[self.con_layers + self.den_layers](x)
        return x   

    



In [29]:
PARAM = {
    "con_layers" : 5,
    "dense_layers" : 1,
    "filter_size" : [3] * 5,
    "output_activation" : "softmax", 
    "dense_output_list" : [32],
    "filter_num" : 16,
    "activation" : "ReLU",
    "filter_org" : "same", #double half
    "input_channel" : 3,
    "padding" : 0,
    "stride" : 2,
    "pool_padding" : 0,
    "pool_stride" : 1,
    "image_size" : 256,
    "pool_filter_size" : 3,
    "batch_size" : 128,
    "eta" : 0.0001,
    "dropout" : [0.2] * 6,
    "epochs" : 5
}

net = ConvolutionalNeuralNetwork(PARAM)
net = net.to(device)
print("***********")
total = 0
for p in net.parameters():
    print(p.numel())
    total += p.numel()



print("Total Parameters", total)


125
60
27
11
3
Computation ::  18213994
***********
432
16
2304
16
2304
16
2304
16
2304
16
4608
32
320
10
Total Parameters 14698


In [14]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr = 0.0001)
# epochs = 1
# labels, train_loader, val_loader, test_loader = load_data(128, PARAM["image_size"])
# for epoch in range(epochs):
#     net.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     count = 0
#     # print("image is loading")
#     for images, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = net(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         # print(outputs)
        
#         # Calculate accuracy
#         _, predicted = torch.max(outputs.data, 1)
#         # print(_, predicted)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         count += 1
#         break
    
#     print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Accuracy: {100 * correct / total}%")


image is loading
image is loaded
Epoch 1, Loss: 0.0365389036753821, Accuracy: 9.375%
image is loading
image is loaded
Epoch 2, Loss: 0.03653850252666171, Accuracy: 9.375%
image is loading
image is loaded
Epoch 3, Loss: 0.036538101377941314, Accuracy: 9.375%
image is loading
image is loaded
Epoch 4, Loss: 0.0365377077980647, Accuracy: 9.375%
image is loading
image is loaded
Epoch 5, Loss: 0.0365373104337662, Accuracy: 9.375%
image is loading
image is loaded
Epoch 6, Loss: 0.03653691306946769, Accuracy: 9.375%
image is loading
image is loaded
Epoch 7, Loss: 0.036536519489591086, Accuracy: 9.375%
image is loading
image is loaded
Epoch 8, Loss: 0.036536133478558254, Accuracy: 9.375%
image is loading
image is loaded
Epoch 9, Loss: 0.03653573611425975, Accuracy: 9.375%
image is loading
image is loaded
Epoch 10, Loss: 0.03653532739669558, Accuracy: 9.375%


In [30]:
def train_model(model, device, PARAM):
    wandb.init(project='DL_Assignment2')
    wandb.run.name = 'SAMPLE-RUN'
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), PARAM["eta"])
    labels, train_loader, val_loader, test_loader = load_data(PARAM["batch_size"], PARAM["image_size"])
    for epoch in range(PARAM["epochs"]):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        count = 0
        for images, labels in train_loader:
            # print("image is loaded")
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            # print(outputs)
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            # print(_, predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Accuracy: {100 * correct / total}%")

        wandb.log(
            {
                'epochs' : epoch,
                'training_loss' : running_loss/len(train_loader),
                'training_accuracy' : 100 * correct / total 
            }
        )

        wandb.finish()
        

In [31]:
train_model(net, device, PARAM)


epochs,▁▂▃▃▄▅▆▆▇█
training_accuracy,▁▁▂▁▅▇███▇
training_loss,████▆▄▃▂▂▁
epochs,9
training_accuracy,14.3
training_loss,2.28098


Epoch 1, Loss: 2.3025991954500715, Accuracy: 10.2625%
Epoch 2, Loss: 2.3026319269150024, Accuracy: 9.7375%
Epoch 3, Loss: 2.302504289717901, Accuracy: 9.9125%
Epoch 4, Loss: 2.3025629974546886, Accuracy: 10.2125%
Epoch 5, Loss: 2.302554274362231, Accuracy: 10.1625%
